In [1]:
import requests
import json
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
import pickle
import gzip

In [2]:
url ='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
address = "0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0"

string = """
        {token (id: "%s"){tokenDayData { priceUSD date } } }
        """ % address

r = requests.post(url, json={'query': string})
json_data = json.loads(r.text)

In [3]:
json_data

{'data': {'token': {'tokenDayData': [{'date': 1620172800,
     'priceUSD': '0.8190626828995727871142322608907779'},
    {'date': 1620259200, 'priceUSD': '0.7598982820101466333255290192714618'},
    {'date': 1620345600, 'priceUSD': '0.7443041919106500578273067560668887'},
    {'date': 1620432000, 'priceUSD': '0.8583194177902746432712258589341726'},
    {'date': 1620518400, 'priceUSD': '0.9703325062989384170928540621511288'},
    {'date': 1620604800, 'priceUSD': '0.8866586816998281144846154552284741'},
    {'date': 1620691200, 'priceUSD': '1.079741579592621242068911997102309'},
    {'date': 1620777600, 'priceUSD': '1.021927486311967819061248572265125'},
    {'date': 1620864000, 'priceUSD': '1.038396402294325856871504230701743'},
    {'date': 1620950400, 'priceUSD': '1.492767518693125252910888761698836'},
    {'date': 1621036800, 'priceUSD': '1.617193131329003128375901500669312'},
    {'date': 1621123200, 'priceUSD': '1.71562737583804103674461235909921'},
    {'date': 1621209600, 'priceUS

In [4]:
df_data = json_data['data']['token']['tokenDayData']
df = pd.DataFrame(df_data)

df.priceUSD = df.priceUSD.replace(0, np.nan).dropna()
df.priceUSD = df.priceUSD.astype(float)

last = df.date.iloc[-1:].values[0]
timestep = df.date.iloc[-1:].values[0]-df.date.iloc[-2:-1].values[0]
model = ARIMA(df.priceUSD, order=(1,1,0))
fitted = model.fit()

# Forecast
fc = fitted.forecast(10)

In [6]:
with open('../input/ipfs-cid/bafkreieok3gxhtdx4jmw6qzhr3oayionjp3xqz5ymbkubxvn4sohlrurkm', 'rb') as pkl:
    pickle_preds = pickle.load(pkl).forecast(10)

In [7]:
pickle_preds

100    28.993105
101    29.013279
102    29.005426
103    29.008483
104    29.007293
105    29.007756
106    29.007576
107    29.007646
108    29.007619
109    29.007629
Name: predicted_mean, dtype: float64